### Backprojection idea

Variables:
- $M, N, C$ the height, width and number of channels;
- $\boldsymbol{y} \in \mathbb{R}^{MN}$ the observation;
- $\boldsymbol{x} \in \mathbb{R}^{MNC}$ a reconstruction;
- $\boldsymbol{A} \in \mathbb{R}^{MN \times MNC}$ the degradation operator (or forward operator).

#### Usual problem to be solved:

$$\boldsymbol{\hat{x}} = \argmin \frac{1}{2} \| \boldsymbol{Ax} - \boldsymbol{y} \|_2^2 + \lambda \mathcal{R}(\boldsymbol{x})$$

Where the data fidelity's norm is the norm of $\mathbb{R}^{MN}$ (the observation space).


#### Backprojected problem:

$$\boldsymbol{\hat{x}} = \argmin \frac{1}{2} \| \boldsymbol{P} (\boldsymbol{Ax} - \boldsymbol{y}) \|_2^2 + \lambda \mathcal{R}(\boldsymbol{x})$$

Where $\boldsymbol{P} \in \mathbb{R}^{MNC \times MN}$ the backprojection matrix and the norm is now the norm of $\mathbb{R}^{MNC}$ (the reconstruction space).


#### The backprojection matrix

Two possibilities are explored here:

The adjoint of the forward operator:
$$\boldsymbol{P} = \boldsymbol{A^T}$$

The pseudo-inverse of the forward operator:
$$\boldsymbol{P} = \boldsymbol{A^\dagger} = \boldsymbol{A^T} (\boldsymbol{AA^T})^{-1}

In [2]:
import numpy as np
import jax.numpy as jnp
from scico import functional, linop, loss, metric
from scico.optimize import PDHG
from scico.optimize.admm import ADMM, LinearSubproblemSolver

import matplotlib.pyplot as plt

from time import perf_counter

from src.input_initialization import *
from src.forward_operator.operators import cfa_operator
from src.forward_operator.forward_operator import forward_operator
from src.inversions.baseline_method.inversion_baseline import *

### General parameters

In [25]:
INPUT_DIR = 'input/'

CFA = 'sparse_3'
BINNING = CFA == 'quad_bayer'
MAX_ITER = 200
NOISE_LEVEL = 5

### Ground truth and raw acquisition

In [26]:
# input_name = '01690'
# x, spectral_stencil = initialize_input(INPUT_DIR + input_name + '.png')

# cfa_op = cfa_operator(CFA, x.shape, spectral_stencil)
# forward_op = forward_operator([cfa_op])
# mat = forward_op.matrix
# diag_inv = (1 / (mat @ mat.T).diagonal()).reshape(forward_op.output_shape)

# y = np.clip(forward_op.direct(x) + np.random.normal(0, NOISE_LEVEL / 100, forward_op.output_shape), 0, 1)

In [27]:
INPUT_DIR = 'input/balloons_ms/'
spectral_stencil = np.array([i for i in range(400, 701, 10)])
x = np.empty((512, 512, 31))

from os import listdir
for i, file in enumerate(listdir(INPUT_DIR)):
    if file.endswith('.png'):
        x[:, :, int(file[-6:-4]) - 1] = initialize_inverse_input(INPUT_DIR + file)

x /= np.max(x)

cfa_op = cfa_operator(CFA, x.shape, spectral_stencil, 'WV34bands_Spectral_Responses.npz')
forward_op = forward_operator([cfa_op])
mat = forward_op.matrix
diag_inv = (1 / (mat @ mat.T).diagonal()).reshape(forward_op.output_shape)

y = forward_op.direct(x) + np.random.normal(0, NOISE_LEVEL / 100, forward_op.output_shape)

### Data fidelity term (no backprojection)

In [28]:
def forward_pass(x):
    return jnp.array(forward_op.direct(x))

def adjoint_pass(y):
    return jnp.array(forward_op.adjoint(y))

A = linop.LinearOperator(input_shape=x.shape, output_shape=y.shape, eval_fn=forward_pass, adj_fn=adjoint_pass)

f = loss.SquaredL2Loss(y=jnp.array(y), A=A)

### Data fidelity term with adjoint backprojection

In [29]:
def adj_forw_pass(x):
    return jnp.array(forward_op.adjoint(forward_op.direct(x)))

A_adj = linop.LinearOperator(input_shape=x.shape, output_shape=x.shape, eval_fn=adj_forw_pass, adj_fn=adj_forw_pass)
y_adj = forward_op.adjoint(y)

f_adj = loss.SquaredL2Loss(y=jnp.array(y_adj), A=A_adj)

### Data fidelity term with pseudo-inverse backprojection

In [30]:
def pinv_forw_pass(x):
    return jnp.array(forward_op.adjoint(diag_inv * forward_op.direct(x)))

A_pinv = linop.LinearOperator(input_shape=x.shape, output_shape=x.shape, eval_fn=pinv_forw_pass, adj_fn=pinv_forw_pass)
y_pinv = forward_op.adjoint(diag_inv * y)

f_pinv = loss.SquaredL2Loss(y=jnp.array(y_pinv), A=A_pinv)

### Baseline inversion

In [31]:
baseline_inverse = Inverse_problem(CFA, BINNING, 0, x.shape, spectral_stencil)
start = perf_counter()
x_baseline = jnp.array(baseline_inverse(y))
mid_1 = perf_counter()

TypeError: get_sparse_3_mask() missing 1 required positional argument: 'responses_file'

In [ ]:
import optuna

def objective(trial):
    lambd = trial.suggest_float('lambd', 0, 1)

    g = functional.L21Norm(l2_axis=(0, 3))
    C = lambd * linop.FiniteDifference(input_shape=x.shape, append=0, axes=(0, 1))
    C_squared_norm = np.float64(linop.operator_norm(C))**2

    sigma = trial.suggest_int('sigma', 1, 100)
    tmp = trial.suggest_float('tmp', 0, 1)
    tau = tmp / (sigma * C_squared_norm)

    solver_PDHG_pinv = PDHG(
        f=f_pinv,
        g=g,
        C=C,
        tau=tau,
        sigma=sigma,
        x0=x_baseline,
        maxiter=200
    )

    return metric.psnr(x, solver_PDHG_pinv.solve())

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, n_jobs=-1)
best_trial = study.best_trial

for key, value in best_trial.params.items():
    print(key, value)

### TV PDHG (no backprojection)

In [33]:
lambd = 5e-3

g = functional.L21Norm(l2_axis=(0, 3))
C = lambd * linop.FiniteDifference(input_shape=x.shape, append=0, axes=(0, 1))
C_squared_norm = np.float64(linop.operator_norm(C))**2

sigma = 1e2
tau = 0.99 / (sigma * C_squared_norm)

solver_PDHG = PDHG(
    f=f,
    g=g,
    C=C,
    tau=tau,
    sigma=sigma,
    x0=forward_op.adjoint(y),
    maxiter=MAX_ITER
)

/home/matthieu/.local/lib/python3.10/site-packages/scico/optimize/_primaldual.py:181: UserWarning: Argument 1 of 1 is an np.ndarray. Will cast it to DeviceArray. To suppress this warning cast all np.ndarrays to DeviceArray first.
  self.x = ensure_on_device(x0)


### TV PDHG with adjoint backprojection

In [34]:
lambd = 5e-3

g = functional.L21Norm(l2_axis=(0, 3))
C = lambd * linop.FiniteDifference(input_shape=x.shape, append=0, axes=(0, 1))
C_squared_norm = np.float64(linop.operator_norm(C))**2

sigma = 1e2
tau = 0.99 / (sigma * C_squared_norm)

solver_PDHG_adj = PDHG(
    f=f_adj,
    g=g,
    C=C,
    tau=tau,
    sigma=sigma,
    x0=forward_op.adjoint(y),
    maxiter=MAX_ITER
)

### TV PDHG with pseudo-inverse backprojection

In [35]:
lambd = 5e-2

g = functional.L21Norm(l2_axis=(0, 3))
C = lambd * linop.FiniteDifference(input_shape=x.shape, append=0, axes=(0, 1))
C_squared_norm = np.float64(linop.operator_norm(C))**2

sigma = 3e1
tau = 0.4 / (sigma * C_squared_norm)

solver_PDHG_pinv = PDHG(
    f=f_pinv,
    g=g,
    C=C,
    tau=tau,
    sigma=sigma,
    x0=forward_op.adjoint(y),
    maxiter=MAX_ITER
)

### BM3D PnP ADMM (no backprojection)

In [ ]:
lambd = 5e-2

g = lambd * 6e-2 * functional.BM3D(is_rgb=True)
C = linop.Identity(input_shape=x.shape)

rho = 2 * lambd * 10**-1

solver_BM3D = ADMM(
    f=f,
    g_list=[g],
    C_list=[C],
    rho_list=[rho],
    x0=x_baseline,
    maxiter=MAX_ITER // 20,
    subproblem_solver=LinearSubproblemSolver(cg_kwargs={'tol': 1e-3, 'maxiter': 100})
)

### BM3D PnP ADMM with adjoint backprojection

In [ ]:
lambd = 5e-2

g = lambd * 6e-2 * functional.BM3D(is_rgb=True)
C = linop.Identity(input_shape=x.shape)

rho = 2 * lambd * 10**-1

solver_BM3D_adj = ADMM(
    f=f_adj,
    g_list=[g],
    C_list=[C],
    rho_list=[rho],
    x0=x_baseline,
    maxiter=MAX_ITER // 20,
    subproblem_solver=LinearSubproblemSolver(cg_kwargs={'tol': 1e-3, 'maxiter': 100})
)

### BM3D PnP ADMM with pseudo-inverse backprojection

In [ ]:
lambd = 5e-2

g = lambd * 6e-2 * functional.BM3D(is_rgb=True)
C = linop.Identity(input_shape=x.shape)

rho = 2 * lambd * 10**-1

solver_BM3D_pinv = ADMM(
    f=f_pinv,
    g_list=[g],
    C_list=[C],
    rho_list=[rho],
    x0=x_baseline,
    maxiter=MAX_ITER // 20,
    subproblem_solver=LinearSubproblemSolver(cg_kwargs={'tol': 1e-3, 'maxiter': 100})
)

### Running the solvers

In [36]:
mid_2 = perf_counter()
x_PDHG = solver_PDHG.solve()
mid_3 = perf_counter()
x_PDHG_adj = solver_PDHG_adj.solve()
mid_4 = perf_counter()
x_PDHG_pinv = solver_PDHG_pinv.solve()
mid_5 = perf_counter()
# x_BM3D = solver_BM3D.solve()
# mid_6 = perf_counter()
# x_BM3D_adj = solver_BM3D_adj.solve()
# mid_7 = perf_counter()
# x_BM3D_pinv = solver_BM3D_pinv.solve()
# end = perf_counter()

In [37]:
print(metric.psnr(x, np.clip(x_PDHG, 0, 1)))
print(metric.mse(x, np.clip(x_PDHG, 0, 1)))
print(mid_3 - mid_2)

print(metric.psnr(x, np.clip(x_PDHG_adj, 0, 1)))
print(metric.mse(x, np.clip(x_PDHG_adj, 0, 1)))
print(mid_4 - mid_3)

print(metric.psnr(x, np.clip(x_PDHG_pinv, 0, 1)))
print(metric.mse(x, np.clip(x_PDHG_pinv, 0, 1)))
print(mid_5 - mid_4)

20.653252
0.008603494
247.0749602609999
20.651829
0.008606311
401.59159047399953
20.28673
0.009361103
212.626590244


### Plotting the results

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, sharex=True, sharey=True, figsize=(20, 20))
ax[0][0].imshow(x)
ax[0][0].set_title('Reference', size=20)
ax[0][1].imshow(y, cmap='gray')
ax[0][1].set_title(f'Raw image, noise level: {NOISE_LEVEL}', size=20)
ax[0][2].imshow(x_baseline)
ax[0][2].set_title(f'Baseline: {metric.psnr(x, x_baseline):.2f} (dB), {mid_1 - start:.1f}s', size=20)
ax[1][0].imshow(x_PDHG)
ax[1][0].set_title(f'TV: {metric.psnr(x, x_PDHG):.2f} (dB), {mid_3 - mid_2:.1f}s', size=20)
ax[1][1].imshow(x_PDHG_adj)
ax[1][1].set_title(f'TV adj backproj: {metric.psnr(x, x_PDHG_adj):.2f} (dB), {mid_4 - mid_3:.1f}s', size=20)
ax[1][2].imshow(x_PDHG_pinv)
ax[1][2].set_title(f'TV pinv backproj: {metric.psnr(x, x_PDHG_pinv):.2f} (dB), {mid_5 - mid_4:.1f}s', size=20)
# ax[2][0].imshow(x_BM3D)
# ax[2][0].set_title(f'PnP: {metric.psnr(x, x_BM3D):.2f} (dB), {mid_6 - mid_5:.1f}s', size=20)
# ax[2][1].imshow(x_BM3D_adj)
# ax[2][1].set_title(f'PnP adj backproj: {metric.psnr(x, x_BM3D_adj):.2f} (dB), {mid_7 - mid_6:.1f}s', size=20)
# ax[2][2].imshow(x_BM3D_pinv)
# ax[2][2].set_title(f'PnP pinv backproj: {metric.psnr(x, x_BM3D_pinv):.2f} (dB), {end - mid_7:.1f}s', size=20)

# fig.savefig('backproj_comparisons.png')

### Plotting the erros

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3, sharex=True, sharey=True, figsize=(20, 20))
ax[0][0].imshow(x)
ax[0][0].set_title('Reference', size=20)
ax[0][1].imshow(y, cmap='gray')
ax[0][1].set_title(f'Raw image, noise level: {NOISE_LEVEL}', size=20)
ax[0][2].imshow(x -x_baseline)
ax[0][2].set_title(f'Baseline: {metric.psnr(x, x_baseline):.2f} (dB), {mid_1 - start:.1f}s', size=20)
ax[1][0].imshow(x -x_PDHG)
ax[1][0].set_title(f'TV: {metric.psnr(x, x_PDHG):.2f} (dB), {mid_3 - mid_2:.1f}s', size=20)
ax[1][1].imshow(x -x_PDHG_adj)
ax[1][1].set_title(f'TV adj backproj: {metric.psnr(x, x_PDHG_adj):.2f} (dB), {mid_4 - mid_3:.1f}s', size=20)
ax[1][2].imshow(x -x_PDHG_pinv)
ax[1][2].set_title(f'TV pinv backproj: {metric.psnr(x, x_PDHG_pinv):.2f} (dB), {mid_5 - mid_4:.1f}s', size=20)
# ax[2][0].imshow(x -x_BM3D)
# ax[2][0].set_title(f'PnP: {metric.psnr(x, x_BM3D):.2f} (dB), {mid_6 - mid_5:.1f}s', size=20)
# ax[2][1].imshow(x -x_BM3D_adj)
# ax[2][1].set_title(f'PnP adj backproj: {metric.psnr(x, x_BM3D_adj):.2f} (dB), {mid_7 - mid_6:.1f}s', size=20)
# ax[2][2].imshow(x -x_BM3D_pinv)
# ax[2][2].set_title(f'PnP pinv backproj: {metric.psnr(x, x_BM3D_pinv):.2f} (dB), {end - mid_7:.1f}s', size=20)

# fig.savefig('backproj_comparisons_errors.png')